In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math



df = pd.read_csv('Rotman MMA Summer Datathon.csv')
#filtering for data for women only
women_data = df[df['team_name'].str.contains("Women") & df['team_name'].str.contains("Canada")]
# print (women_data.columns)
#women_data.columns

#{name:[power score , penalty score]}
player_score = dict()


def div_safe(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 0
    
def cal_Distance(x1,y1,x2,y2):
    return round(math.sqrt((x2-x1)**2+(y2-y1)**2),2)


In [2]:
#------- Power Sepecialist----------


# {name : [count of t, total count, receive count of t ,  total recieve, [distance list] , [location list[x,y]]}
power_PlayCounts = dict()
# {name : [play success rate, receive success rate ,  average distance, count of edge pass}
power_PlayStats = dict()
# {name : [count of successful shot, total shot count, long distance shot count , short distance shot count]}
power_ShotCounts = dict()
# {name : [shot success rate]}
power_ShotStats = dict()
#{name : [play success rate, receive success rate ,  average distance}
power_Score = dict()


#Extract data from the sources and orgnaize it in dictionary
for i , j in women_data.iterrows():
    # field of interest
    name = women_data.loc[i]['player_name']
    receiver = women_data.loc[i]['receiver_name']
    
    situation = women_data.loc[i]['situation_type']
    event = women_data.loc[i]['event_type']
    success = women_data.loc[i]['event_successful']
    shot = women_data.loc[i]['shot_type']
    
    eventX = women_data.loc[i]['x_event']
    eventY = women_data.loc[i]['y_event']
    receiveX = women_data.loc[i]['receiver_x']
    receiveY = women_data.loc[i]['receiver_y']
    if situation == '5 on 4' or situation == '5 on 3' or situation == '4 on 3' or situation == '6 on 5' or situation == '6 on 4':
        if event == 'Play':        
            if name not in power_PlayCounts:
                power_PlayCounts[name] = [0,0,0,0,[],[]]
            if receiver not in power_PlayCounts: 
                power_PlayCounts[receiver] = [0,0,0,0,[],[]]

            power_PlayCounts[name][1] = power_PlayCounts[name][1] + 1
            power_PlayCounts[receiver][3] = power_PlayCounts[receiver][3] + 1
            
            if success == 't':
                power_PlayCounts[name][0] = power_PlayCounts[name][0] + 1
                power_PlayCounts[receiver][2] = power_PlayCounts[receiver][2] + 1
                power_PlayCounts[name][4].append(cal_Distance(eventX,eventY,receiveX,receiveY))
                power_PlayCounts[name][5].append([eventX,eventY])
        
        elif event == 'Shot':               
            if name not in power_ShotCounts:
                power_ShotCounts[name] = [0,0,0,0]
                
            power_ShotCounts[name][1] = power_ShotCounts[name][1]+1         
            
            if success == 't':

                power_ShotCounts[name][0] = power_ShotCounts[name][0]+1
                
                if shot == 'Snapshot' or shot == 'Slapshot' or shot == 'Wristshot':
                    power_ShotCounts[name][2] = power_ShotCounts[name][2]+1
                
                elif shot == 'Wrap Around' or shot == 'Deflection':
                    power_ShotCounts[name][3] = power_ShotCounts[name][3]+1
           
# calculate Status
for name,value in power_PlayCounts.items():
    #play success rate
    power_PlayStats[name] = []
    power_PlayStats[name].append(div_safe(value[0],value[1]))
    #recieve success rate
    power_PlayStats[name].append(div_safe(value[2],value[3]))
    #avg distance of success pass
    power_PlayStats[name].append(div_safe(sum(value[4]),len(value[4])))

    
for name,value in power_ShotCounts.items():
    #shot success rate
    power_ShotStats[name] = []
    power_ShotStats[name].append(value[0])
    power_ShotStats[name].append(div_safe(value[0],value[1]))    
    power_ShotStats[name].append(value[2])
    power_ShotStats[name].append(value[3])
    


# print ("All Power: ", power_PlayCounts)
# print ("------------------------")
# print ("Stats: ", power_PlayStats)
# print ("============")
# print ("All Power Play shot: ", power_ShotCounts)
# print ("------------------------------------------------------------")
# print ("Shot Success Rate: ", power_ShotStats)
    




for i in range (3):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(power_PlayStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in power_Score:
            power_Score[name] = [0,0,0,0,0,0,0]
        if temp_value != value[i]:
            power_Score[name][i] = max (100 - j*5, 0)
            temp_score = power_Score[name][i]
            j += 1
        else:
            power_Score[name][i] = temp_score
        temp_value = value[i]    
#         print ("Play: ",i, name, value, power_Score[name][i])
#     print ("------------")

for i in range (4):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(power_ShotStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in power_Score:
            power_Score[name] = [0,0,0,0,0,0,0]
        if temp_value != value[i]:
            power_Score[name][i+3] = max (100 - j*5, 0)
            temp_score = power_Score[name][i+3]
            j += 1
        else:
            power_Score[name][i+3] = temp_score
        temp_value = value[i]  
#         print ("Shot: ",i, name, value, power_Score[name][i+3])
#     print ("------------")





#{25%, 10%, 15%, 10%, 20%, 10%, 10%}     
# {name : [play success rate, receive success rate ,average distance,# total count, # at def side]}
# print ("===========================")
# print ("power_Socre: ",power_Score)


#assigne total score
for name,value in power_Score.items(): 
    if name not in player_score:
        player_score[name] = [0,0]
    player_score[name][0] = value[0]*0.25+value[1]*0.10+value[2]*0.15+value[3]*0.10+value[4]*0.20+value[5]*0.10+value[6]*0.10
    
    
print ("==================")
print (sorted(player_score.items(), key=lambda e: e[1][0],reverse = True))


[('Lauriane Rougeau', [91.0, 0]), ('Marie-Philip Poulin', [84.5, 0]), ('Brigette Lacquette', [79.0, 0]), ('Jocelyne Larocque', [78.5, 0]), ('Meaghan Mikkelson', [74.5, 0]), ('Laura Fortino', [73.75, 0]), ('Meghan Agosta', [73.75, 0]), ('Laura Stacey', [71.5, 0]), ('Erin Ambrose', [71.0, 0]), ('Brianne Jenner', [70.75, 0]), ('Loren Gabel', [70.5, 0]), ('Jillian Saulnier', [69.75, 0]), ('Ann-Sophie Bettez', [67.5, 0]), ('Renata Fast', [67.25, 0]), ('Haley Irwin', [65.75, 0]), ('Melodie Daoust', [65.25, 0]), ('Sarah Nurse', [62.5, 0]), ('Emily Clark', [60.25, 0]), ('Blayre Turnbull', [59.75, 0]), ('Rebecca Johnston', [59.5, 0]), ('Jennifer Wakefield', [57.5, 0]), ('Natalie Spooner', [56.5, 0]), ('Bailey Bram', [54.75, 0]), ('Halli Krzyzaniak', [54.75, 0]), ('Jamie Lee Rattray', [49.25, 0]), ('Victoria Bach', [41.0, 0]), ('Jamie Bourbonnais', [35.5, 0]), ('Ann-Renee Desbiens', [25.0, 0]), ('Shannon Szabados', [25.0, 0]), ('Genevieve Lacasse', [25.0, 0]), ('Rebecca Leslie', [25.0, 0]), ('Em

In [3]:
#------- Penalty Sepecialist----------


# {name : [count of t, total count, receive count of t ,  total recieve, [distance list] , [location list[x,y]]}
penalty_PlayCounts = dict()
# {name : [total count, [game,time] , [location list[x,y]]}
penalty_TakeawayCounts = dict()
# {name : [play success rate, receive success rate ,  average distance, count of edge pass}
penalty_PlayStats = dict()
# {name : [# total count, # at def side]}
penalty_TakeawayStats = dict()
# {name : [play success rate, receive success rate ,average distance,# total count, # at def side]}
penalty_Score = dict()


#Extract data from the sources and orgnaize it in dictionary
for i , j in women_data.iterrows():
    # field of interest
    name = women_data.loc[i]['player_name']
    receiver = women_data.loc[i]['receiver_name']
    
    situation = women_data.loc[i]['situation_type']
    game = women_data.loc[i]['game_name']
    time = women_data.loc[i]['event_time']
    
    event = women_data.loc[i]['event_type']
    success = women_data.loc[i]['event_successful']
    
    eventX = women_data.loc[i]['x_event']
    eventY = women_data.loc[i]['y_event']
    receiveX = women_data.loc[i]['receiver_x']
    receiveY = women_data.loc[i]['receiver_y']
    
    
    if situation == '4 on 5' or situation == '3 on 5' or situation == '3 on 4':
        if event == 'Play':               
            if name not in penalty_PlayCounts:
                penalty_PlayCounts[name] = [0,0,0,0,[],[]]
            if receiver not in penalty_PlayCounts: 
                penalty_PlayCounts[receiver] = [0,0,0,0,[],[]]

            penalty_PlayCounts[name][1] = penalty_PlayCounts[name][1] + 1
            penalty_PlayCounts[receiver][3] = penalty_PlayCounts[receiver][3] + 1
            if success == 't':
                penalty_PlayCounts[name][0] = penalty_PlayCounts[name][0] + 1
                penalty_PlayCounts[receiver][2] = penalty_PlayCounts[receiver][2] + 1
                penalty_PlayCounts[name][4].append(cal_Distance(eventX,eventY,receiveX,receiveY))
                penalty_PlayCounts[name][5].append([eventX,eventY])
                
        elif event == 'Takeaway':
            if name not in penalty_TakeawayCounts:
                penalty_TakeawayCounts[name] = [0,[],[]]
            penalty_TakeawayCounts[name][0] = penalty_TakeawayCounts[name][0] + 1
            penalty_TakeawayCounts[name][1].append([game,time])
            penalty_TakeawayCounts[name][2].append([eventX,eventY])

            
            
# calculate Status
for name,value in penalty_PlayCounts.items():
    #--------------------play ----------------------
    penalty_PlayStats[name] = []
    #play success rate
    penalty_PlayStats[name].append(div_safe(value[0],value[1]))
    #recieve success rate
    penalty_PlayStats[name].append(div_safe(value[2],value[3]))
    #avg distance of success pass
    penalty_PlayStats[name].append(div_safe(sum(value[4]),len(value[4])))
    
for name,value in penalty_TakeawayCounts.items():    
    #--------------------takeaway ----------------------
    penalty_TakeawayStats[name] = []
    #total count 
    penalty_TakeawayStats[name].append(value[0])
    penalty_TakeawayStats[name].append(0)
    for i in penalty_TakeawayCounts[name][2][0]:
        if i < 100:
            penalty_TakeawayStats[name][1] += 1
    
    
# print ("All Play: ", penalty_PlayCounts)
# print ("------------------------")
# print ("Stats: ", penalty_PlayStats)
# print ("===================================")
# print ("All Takeaway: ", penalty_TakeawayCounts)
# print ("------------------------")
# print ("Stats: ", penalty_TakeawayStats)



#assign scores for play
for i in range (3):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(penalty_PlayStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in penalty_Score:
            penalty_Score[name] = [0,0,0,0,0]
            
        if temp_value != value[i]:
            penalty_Score[name][i] = max (100 - j*5, 0)
            temp_score = penalty_Score[name][i]
            j += 1
        else:
            penalty_Score[name][i] = temp_score
        temp_value = value[i]        
#         print (i, name, value, penalty_Score[name][i])
#     print ("------------")

#assign scores for takeaway
for i in range (2):
    j = 0
    temp_value = 0
    temp_score = 100
    for name, value in sorted(penalty_TakeawayStats.items(), key=lambda e: e[1][i],reverse = True):
        if name not in penalty_Score:
            penalty_Score[name] = [0,0,0,0,0]
            
        if temp_value != value[i]:
            penalty_Score[name][i+3] = max (100 - j*5, 0)
            temp_score = penalty_Score[name][i+3]
            j += 1
        else:
            penalty_Score[name][i+3] = temp_score
        temp_value = value[i]  
#         print (i, name, value, penalty_Score[name][i+3])
#     print ("------------")
    
    
# print ("==================")
#{30%, 15%, 5%, 35%, 15%}     
# {name : [play success rate, receive success rate ,average distance,# total count takeaway, # at def side for takeaway]}
# print ("penalty_Score: ",penalty_Score)


#assigne total score
for name,value in penalty_Score.items(): 
    if name not in player_score:
        player_score[name] = [0,0]
    player_score[name][1]=value[0]*0.30+value[1]*0.15+value[2]*0.05+value[3]*0.35+value[4]*0.15
    
    
print ("==================")
print (sorted(player_score.items(), key=lambda e: e[1][1],reverse = True))

[('Jocelyne Larocque', [78.5, 95.25]), ('Erin Ambrose', [71.0, 92.0]), ('Halli Krzyzaniak', [54.75, 90.75]), ('Laura Stacey', [71.5, 88.5]), ('Rebecca Johnston', [59.5, 86.0]), ('Jillian Saulnier', [69.75, 85.25]), ('Emily Clark', [60.25, 85.0]), ('Blayre Turnbull', [59.75, 83.75]), ('Renata Fast', [67.25, 83.0]), ('Marie-Philip Poulin', [84.5, 82.25]), ('Laura Fortino', [73.75, 81.75]), ('Natalie Spooner', [56.5, 76.25]), ('Brigette Lacquette', [79.0, 74.5]), ('Brianne Jenner', [70.75, 71.75]), ('Victoria Bach', [41.0, 43.25]), ('Lauriane Rougeau', [91.0, 41.75]), ('Meaghan Mikkelson', [74.5, 41.25]), ('Meghan Agosta', [73.75, 40.5]), ('Shannon Szabados', [25.0, 35.0]), ('Ann-Sophie Bettez', [67.5, 33.25]), ('Bailey Bram', [54.75, 33.25]), ('Sarah Nurse', [62.5, 29.75]), ('Mich Zandee-Hart', [0, 26.5]), ('Ann-Renee Desbiens', [25.0, 0]), ('Genevieve Lacasse', [25.0, 0]), ('Jamie Lee Rattray', [49.25, 0]), ('Rebecca Leslie', [25.0, 0]), ('Jamie Bourbonnais', [35.5, 0]), ('Loren Gabel',